In [482]:
import pandas as pd
import os
import datetime as dt
import numpy as np
from datetime import timedelta
import sqlite3
%run C:\Users\sgasmi\Notebooks\Column_autosize.ipynb

## Fusion des fichiers

In [419]:
folder= r"C:\Users\sgasmi\Desktop\Extractions\Datahome files"

In [420]:
conso = pd.DataFrame()

for root, dirs, files in os.walk(folder):
    for filename in files:
        location = folder + '/' + filename
        if conso.empty :
            conso = pd.read_excel(location)
        else :
            df = pd.read_excel(location)
            conso = conso.append(df,ignore_index=True)
        print(filename)

DATA_HOME_28022021.xlsx
DATA_HOME_30042021.xlsx
DATA_HOME_31012021.xlsx
DATA_HOME_31032021.xlsx


## Préparation des données

### Convertion des dates

In [421]:
conso["Report Date"] = pd.to_datetime(conso["Report Date"],format='%d/%m/%Y')
conso["Headcount"] = 1

In [422]:
conso[["Legal entity", "Site"]] = conso[["Legal entity", "Site"]].fillna("Vide")

## Création des dimensions

### Dimension état civil

In [423]:
dim_civil_prep = conso[["MyHR ID", "First name", "Last name", "Birth date", "Sex"]].drop_duplicates().copy()

In [424]:
dim_civil_prep = dim_civil_prep.reset_index(drop=True)
fact_head_prep["merge"] = 1

In [425]:
print(len(conso["MyHR ID"].unique()))
print(len(dim_civil_prep))

15640
15665


### Gestion des modifications de données

In [426]:
doublons_civil = dim_civil_prep.groupby(["MyHR ID"])["MyHR ID"].count().to_frame(name = 'doublons').reset_index()

In [427]:
doublons_mat = doublons_civil.loc[doublons_civil["doublons"]>1, "MyHR ID"]

In [428]:
doublons_civil = dim_civil_prep.loc[dim_civil_prep["MyHR ID"].isin(doublons_mat)].sort_values(["MyHR ID"])

In [429]:
civil_key = ["MyHR ID", "First name", "Last name", "Birth date", "Sex"]
to_get = ["MyHR ID", "First name", "Last name", "Birth date", "Sex", "Report Date"]

In [430]:
doublons_civil = doublons_civil.merge(conso[to_get], on=civil_key, how="left")

In [431]:
doublons_civil = doublons_civil.groupby(["MyHR ID"])["Report Date"].max().reset_index()

In [432]:
civil_to_add = doublons_civil.merge(conso[to_get], on=["MyHR ID", "Report Date"], how="inner")

In [433]:
civil_to_add = civil_to_add.drop(["Report Date"], axis=1)

In [434]:
dim_civil_prep = dim_civil_prep.loc[dim_civil_prep["MyHR ID"].isin(doublons_mat)==False]

In [435]:
dim_civil_prep = dim_civil_prep.append(civil_to_add)

In [436]:
dim_civil_prep = dim_civil_prep.drop_duplicates()

In [437]:
dim_civil_prep = dim_civil_prep.reset_index(drop=True)
dim_civil_prep["key_civil"] = dim_civil_prep.index

In [438]:
dim_civil = dim_civil_prep.copy()

In [439]:
dim_civil["Merge"]=1

### Dimension contrat

In [440]:
contract_keys= ["MyHR ID", "Contract Type", "Contract start date", "Contract end date", "Group Seniority", "Site", "Legal entity"]
to_keep = contract_keys.copy()
to_keep.append("Report Date")

In [441]:
dim_contract_prep = conso[contract_keys].drop_duplicates().copy()

In [442]:
dim_contract_prep = dim_contract_prep.merge(conso[to_keep], on=contract_keys, how="left")

In [443]:
dim_contract_prep.dtypes

MyHR ID                        object
Contract Type                  object
Contract start date    datetime64[ns]
Contract end date      datetime64[ns]
Group Seniority        datetime64[ns]
Site                           object
Legal entity                   object
Report Date            datetime64[ns]
dtype: object

In [444]:
end_date = pd.to_datetime("2050-01-01", format="%Y/%m/%d")
dim_contract_prep.loc[dim_contract_prep["Contract end date"].isna()==True, "Contract end date" ] = end_date

In [445]:
dim_contract_prep = dim_contract_prep.groupby(contract_keys)["Report Date"].min().reset_index()

In [446]:
dbl = dim_contract_prep.groupby("MyHR ID")["MyHR ID"].count().to_frame(name = 'doublons').reset_index()

In [447]:
dim_contract_prep = dim_contract_prep.sort_values(["MyHR ID", "Report Date"])

In [448]:
dim_contract_prep[dim_contract_prep["MyHR ID"]=="AAB086"]

,MyHR ID,Contract Type,Contract start date,Contract end date,Group Seniority,Site,Legal entity,Report Date
58,AAB086,FT3+,2018-03-19,2021-03-18,2018-03-19,CHN_BEIJING,FRA-BEIJ,2021-01-31
59,AAB086,FT3+,2021-03-19,2024-03-18,2018-03-19,CHN_BEIJING,FRA-BEIJ,2021-03-31


In [449]:
dim_contract_prep["End date"] = ""
for i, val in enumerate(dim_contract_prep.iterrows()):
    if i < len(dim_contract_prep)-1:
        if dim_contract_prep.at[i, "MyHR ID"] == dim_contract_prep.at[i+1, "MyHR ID"]:
            dim_contract_prep.at[i, "End date"] = dim_contract_prep.at[i+1, "Report Date"] - timedelta(days=1)
        else:
            dim_contract_prep.at[i, "End date"] = end_date
    else :
        dim_contract_prep.at[i, "End date"] = end_date

In [450]:
dim_contract_prep["End date"] = pd.to_datetime(dim_contract_prep["End date"])

In [451]:
dim_contract_prep = dim_contract_prep.reset_index(drop=True)
dim_contract_prep["key_contract"] = dim_contract_prep.index

In [452]:
dim_contract_prep[dim_contract_prep["MyHR ID"]=="AAB086"]

,MyHR ID,Contract Type,Contract start date,Contract end date,Group Seniority,Site,Legal entity,Report Date,End date,key_contract
58,AAB086,FT3+,2018-03-19,2021-03-18,2018-03-19,CHN_BEIJING,FRA-BEIJ,2021-01-31,2021-03-30,58
59,AAB086,FT3+,2021-03-19,2024-03-18,2018-03-19,CHN_BEIJING,FRA-BEIJ,2021-03-31,2050-01-01,59


In [476]:
dim_contract = dim_contract_prep.copy()

In [477]:
dim_contract = dim_contract.rename({"Report Date":"Beg_Date","End date": "End_Date", "MyHR ID" : "MyHR_ID"}, axis=1)

## Création de la dimension statut

In [455]:
statut_fields= ["MyHR ID", "Status", "Category"]
to_keep = statut_fields.copy()
to_keep.append("Report Date")

In [456]:
dim_statut_prep = conso[statut_fields].drop_duplicates().reset_index(drop=True).copy()

In [457]:
dim_statut_prep = dim_statut_prep.merge(conso[to_keep], on = statut_fields, how="left")

In [458]:
dim_statut_prep = dim_statut_prep.reset_index(drop=True)

In [459]:
dim_statut_prep = dim_statut_prep.groupby(statut_fields)["Report Date"].min().reset_index()

In [460]:
dim_statut_prep["End date"] = ""
for i, val in enumerate(dim_statut_prep.iterrows()):
    if i < len(dim_statut_prep)-1:
        if dim_statut_prep.at[i, "MyHR ID"] == dim_statut_prep.at[i+1, "MyHR ID"]:
            dim_statut_prep.at[i, "End date"] = dim_statut_prep.at[i+1, "Report Date"] - timedelta(days=1)
        else:
            dim_statut_prep.at[i, "End date"] = end_date
    else :
        dim_statut_prep.at[i, "End date"] = end_date

In [461]:
dim_statut_prep["End date"] = pd.to_datetime(dim_statut_prep["End date"])

In [462]:
dim_statut_prep = dim_statut_prep.reset_index(drop=True)
dim_statut_prep["Key_Statut"] = dim_statut_prep.index

In [463]:
dim_statut_prep = dim_statut_prep.rename({"Report Date":"Beg_Date","End date": "End_Date", "MyHR ID" : "MyHR_ID"}, axis=1)

## Création de la table temps

In [498]:
dim_temps = pd.DataFrame(columns=["key_temps", "Report Date"])
dim_temps["Report Date"] = conso["Report Date"].drop_duplicates()
dim_temps= dim_temps.sort_values("Report Date")
dim_temps = dim_temps.reset_index(drop=True)
dim_temps["key_temps"] = dim_temps.index
dim_temps["Report Date"] = pd.to_datetime(dim_temps["Report Date"])
dim_temps["Merge"]=1

## Création de la table de fait headcount

In [465]:
fact_head_prep = dim_civil[["MyHR ID", "key_civil", "Merge"]].copy()

In [466]:
fact_head_prep = fact_head_prep.merge(dim_temps, on="Merge", how="outer")

In [467]:
fact_head_prep = fact_head_prep.drop(columns="Merge")

In [468]:
fact_head_prep = fact_head_prep.merge(conso[["MyHR ID", "Report Date", "Headcount"]], on= ["MyHR ID", "Report Date"], how="left")

In [469]:
fact_head_prep = fact_head_prep.reset_index(drop=True)

In [470]:
fact_head_prep.dtypes

MyHR ID                object
key_civil               int64
key_temps               int64
Report Date    datetime64[ns]
Headcount             float64
dtype: object

In [471]:
fact_head_prep["Headcount"]= fact_head_prep["Headcount"].fillna(0)

In [472]:
fact_head_prep = fact_head_prep.rename({"Report Date" :"Report_Date","MyHR ID" : "MyHR_ID"}, axis=1)

In [473]:
def date_merge(source_table, target_table, source_functional_key, target_functional_key, source_date_key,  target_column_to_get, target_date_columns_name=[]):
    # Fonction servant à récupérer une colonne à l'aide d'une clé fonctionnelle et d'un date between. Cette fonction complète une lacune de la bibliothèque Pandas
    # Il est nécessaire d'importer la bibliothèque sqlite3
    
    # source table => Dataframe source qui va être alimenté
    # target_table => Dataframe servant à alimenter le dataframe source
    # source_functional_key => la clé fonctionnelle servant à créer la jointure
    # target_functionnal_key=> la clé fonctionnelle cible servant à créer la jointure en dehors des dates
    # source_date_key => la date provenant du DataFrame source servant à la condition in beetween
    # target_column_to_get => la colonne à récupérer
    # target_date_columns_name => le nom des colonnes à récupérer sans espace, remplacer les espace par "_", ex: toto titi, toto_titi / Ces colonnes doivent être des pandas datetimes
    
    conn = sqlite3.connect(':memory:')
    source_table.to_sql('source', conn, index=False)
    target_table.to_sql('target', conn, index=False)
    
    #Création de la requète
    req = "PRAGMA table_info(source);"
    col_source_prep = pd.read_sql_query(req, conn)
    col_source_prep = list(col_source_prep["name"])
    col_source = []
    
    for val in col_source_prep :
        col_source.append('source.' + str(val)) 
        
    col_source = str(col_source)
    col_source = col_source.replace("[", "")
    col_source = col_source.replace("]", "")
    col_source = col_source.replace("'","")

    req = f'Select {col_source}, {target_column_to_get} from source left join target on source.{source_functional_key}=target.{target_functional_key} and source.{source_date_key} between {target_date_columns_name[0]} and {target_date_columns_name[1]}'
    
    df = pd.read_sql_query(req, conn)
    
    return df

In [478]:
dim_contract.columns

Index(['MyHR_ID', 'Contract Type', 'Contract start date', 'Contract end date',
       'Group Seniority', 'Site', 'Legal entity', 'Beg_Date', 'End_Date',
       'key_contract'],
      dtype='object')

In [483]:
fact_head_prep = date_merge(source_table=fact_head_prep, target_table=dim_contract[["MyHR_ID", "key_contract", "Beg_Date", "End_Date"]], source_functional_key="MyHR_ID", target_functional_key= "MyHR_ID", source_date_key="Report_Date", target_date_columns_name=["Beg_Date","End_Date"], target_column_to_get="key_contract")

In [492]:
resize_column([fact_head_prep], "Table de fait", r"C:\Users\sgasmi\Desktop\Export Python\Table de faits.xlsx")
resize_column([dim_contract], "Dim contract", r'C:\Users\sgasmi\Desktop\Export Python\dim_contract.xlsx' )

In [491]:
fact_head_prep.columns

Index(['MyHR_ID', 'key_civil', 'key_temps', 'Report_Date', 'Headcount',
       'key_contract'],
      dtype='object')